<a href="https://colab.research.google.com/github/still3901/Deep-Learning/blob/main/%EC%8B%A4%EC%8A%B5_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 문장 유사도를 이용한 ChatBot 개발

In [1]:
### 필요한 라이브러리 설치
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

## 데이터 불러오기

In [2]:
### 필요한 라이브러리 임폴트
import pandas as pd

In [3]:
### 챗봇 생성용 데이터 불러오기

# 파일경로설정
file_path = '/content/drive/MyDrive/KDT/딥러닝/자연어처리/ChatBotData.csv'

# 파일 불러오기
df = pd.read_csv(file_path)

# 결과 확인하기
# print(df)
df

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [4]:
### 각 컬럼별 누락 데이터의 수 확인
num_nulls = df.isnull().sum()
print(f'각 컬럼별 누락 데이터의 수 확인 : \n{num_nulls}')

각 컬럼별 누락 데이터의 수 확인 : 
Q        0
A        0
label    0
dtype: int64


## 문장 임베딩 행렬 생성

In [4]:
### 필요한 함수 임폴트
from sentence_transformers import SentenceTransformer
import numpy as np

In [5]:
### 사전 학습된 한국어 모델 다운로드
model_name = 'ddobokki/klue-roberta-base-nli-sts'
model = SentenceTransformer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/576 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
### 문장 임베딩 행렬 생성 -> .values를 사용하여 리스트 형태로 만들어도 된다.

# 0 칼럼 인덱싱
#sentences= df.loc[:,'Q'].values
sentences= df.loc[:,'Q']
print(sentences)
print('-'*80)

# 생성된 임베딩 벡터 저장 --> 리스트 생성
embeddings = []

# for문 + model.encode() -> 임베딩 벡터 생성
for sent in sentences:
    sent_embedding = model.encode(sent)
    embeddings.append(sent_embedding)

0                         12시 땡!
1                    1지망 학교 떨어졌어
2                   3박4일 놀러가고 싶다
3                3박4일 정도 놀러가고 싶다
4                        PPL 심하네
                  ...           
11818             훔쳐보는 것도 눈치 보임.
11819             훔쳐보는 것도 눈치 보임.
11820                흑기사 해주는 짝남.
11821    힘든 연애 좋은 연애라는게 무슨 차이일까?
11822                 힘들어서 결혼할까봐
Name: Q, Length: 11823, dtype: object
--------------------------------------------------------------------------------


In [8]:
# 결과 확인하기
print(f'성분 원소로서 임베딩의 백터의 갯수 : {len(embeddings)}개')
print('-'*80)
print(f'전체 임베딩 벡터 중 상위 3개 출력 : \n{embeddings[:3]}')

성분 원소로서 임베딩의 백터의 갯수 : 11823개
--------------------------------------------------------------------------------
전체 임베딩 벡터 중 상위 3개 출력 : 
[array([-1.38743520e-01,  1.51096890e-02,  5.28436482e-01,  7.21492231e-01,
       -3.91226768e-01, -7.02952623e-01, -8.01475823e-01,  4.91262555e-01,
        2.17926558e-02,  3.80374581e-01, -6.92348480e-01, -1.11402869e-01,
        2.16700807e-01,  3.51305604e-01,  2.36636296e-01,  6.80069625e-01,
        1.71046376e-01, -5.12855768e-01, -1.09594762e-01,  6.90380037e-02,
        2.15583201e-02, -7.99720347e-01, -1.68677017e-01,  6.13967478e-01,
        3.97407681e-01,  4.25235122e-01,  5.24643250e-02,  2.25222990e-01,
       -3.69577408e-01, -3.86963874e-01, -6.03431940e-01,  4.99112606e-01,
       -1.14106648e-01, -1.77366123e-01,  1.42669931e-01, -5.48125923e-01,
        4.89641070e-01, -2.73698956e-01,  5.87849319e-01,  3.24973434e-01,
        5.86495213e-02,  7.93394804e-01, -1.63209453e-01,  6.36751577e-02,
       -2.92038977e-01, -4.99974638e-01,

In [11]:
### 생성된 임베딩 벡터의 리스트 데이터 저장 / 리스트나 딕셔너리 혹은 튜플 그대로 저장하고자 할때 pickle을 사용.

# 필요한 함수 임폴트
import pickle

# 저장 경로 설정
file_path='/content/drive/MyDrive/KDT/딥러닝/자연어처리/chat_embedding.p'

# 저장 -->  with open + pickle.dump()
with open(file_path, 'wb') as f:
    pickle.dump(embeddings, f)

In [6]:
### 저장된 임베딩 벡터의 리스트 데이터 불러오기

# 필요한 함수 임폴트
import pickle

# 저장 경로 설정
file_path='/content/drive/MyDrive/KDT/딥러닝/자연어처리/chat_embedding.p'

# 불러오기 -->  with open + pickle.load()
with open(file_path, 'rb') as f:
    loaded_embeddings = pickle.load(f)

# 결과 확인하기 -> 상위 2개만 출력
print(loaded_embeddings[:2])

[array([-1.38743520e-01,  1.51096890e-02,  5.28436482e-01,  7.21492231e-01,
       -3.91226768e-01, -7.02952623e-01, -8.01475823e-01,  4.91262555e-01,
        2.17926558e-02,  3.80374581e-01, -6.92348480e-01, -1.11402869e-01,
        2.16700807e-01,  3.51305604e-01,  2.36636296e-01,  6.80069625e-01,
        1.71046376e-01, -5.12855768e-01, -1.09594762e-01,  6.90380037e-02,
        2.15583201e-02, -7.99720347e-01, -1.68677017e-01,  6.13967478e-01,
        3.97407681e-01,  4.25235122e-01,  5.24643250e-02,  2.25222990e-01,
       -3.69577408e-01, -3.86963874e-01, -6.03431940e-01,  4.99112606e-01,
       -1.14106648e-01, -1.77366123e-01,  1.42669931e-01, -5.48125923e-01,
        4.89641070e-01, -2.73698956e-01,  5.87849319e-01,  3.24973434e-01,
        5.86495213e-02,  7.93394804e-01, -1.63209453e-01,  6.36751577e-02,
       -2.92038977e-01, -4.99974638e-01,  2.44825318e-01, -4.05069232e-01,
       -2.36141250e-01,  2.71784812e-01,  7.33441174e-01,  1.45670660e-02,
       -6.05235279e-01, 

## ChatBot 데이터 생성
- embedding이라는 새로운 컬럼 생성
- 예상 질문에 대한 문장 임베딩 벡터 저장

In [7]:
# 새로운 컬럼 생성, 예상 질문임베딩 벡터
df.loc[:, 'embedding'] = loaded_embeddings

# 결과 확인하기
print(f'전체 데이터프레임 확인 : \n{df}')
print('-'*80)
print(f'embedding 컬럼의 0번째 성분 원소 확인 : \n{df.iloc[0, -1]}')

전체 데이터프레임 확인 : 
                             Q                         A  label  \
0                       12시 땡!                하루가 또 가네요.      0   
1                  1지망 학교 떨어졌어                 위로해 드립니다.      0   
2                 3박4일 놀러가고 싶다               여행은 언제나 좋죠.      0   
3              3박4일 정도 놀러가고 싶다               여행은 언제나 좋죠.      0   
4                      PPL 심하네                눈살이 찌푸려지죠.      0   
...                        ...                       ...    ...   
11818           훔쳐보는 것도 눈치 보임.        티가 나니까 눈치가 보이는 거죠!      2   
11819           훔쳐보는 것도 눈치 보임.             훔쳐보는 거 티나나봐요.      2   
11820              흑기사 해주는 짝남.                    설렜겠어요.      2   
11821  힘든 연애 좋은 연애라는게 무슨 차이일까?  잘 헤어질 수 있는 사이 여부인 거 같아요.      2   
11822               힘들어서 결혼할까봐        도피성 결혼은 하지 않길 바라요.      2   

                                               embedding  
0      [-0.13874352, 0.015109689, 0.5284365, 0.721492...  
1      [0.08192271, 0.30836907, 0.2768094, -0.0601377...  
2 

## 사용자 정의 함수 생성

In [8]:
### 두 문장의 텍스트 유사도 계산 함수 정의

# 필요한 함수 임폴트
from numpy import dot
from numpy.linalg import norm

# 사용자 정의 함수
def cos_sim(A, B):
    return dot(A, B) / (norm(A)*norm(B))

In [27]:
### 사용자 질문에 대한 응답 구현 함수 정의 -> 샘플을 이용한 테스트

'''
1. 사용자의 질문이 들어오면 해당 질문에 대한 문장 임베딩 값을 구한다.
2. 사용자 질문의 문장 임베딩 값과 챗봇 데이터의 예상 질문에 대한 임베딩 컬럼(df.loc[:, 'embedding'])에
저장해둔 예상 질문 샘플들의 문장 임베딩 값들을 이용, 코사인 유사도를 계산
--> 코사인 유사도 값을 저장하는 컬럼(df.loc[:, 'score'])을 생성하고 값을 저장
3. 코사인 유사도 값이 가장 높은 질문 샘플을 찾는다.
4. 해당 질문 샘플과 짝이 되는 답변 샘플을 반환한다.
'''
# tqdm
from tqdm import tqdm

# 사용자 질문 생성
question = '남자친구랑 여행가고 싶다'

# 사용자 질문 -> 임베딩 벡터 생성
user_embedding = model.encode(question)

# 코사인 유사도 계산 -> score 컬럼 생성 -> 코사인 유사도 값 저장
scores = []
for embedding in tqdm(df.loc[:,'embedding']):
    score = cos_sim(user_embedding, embedding)
    scores.append(score)
# print(scores)

# score 컬럼 생성 => 코사인 유사도 값을 저장
df.loc[:,'score'] = scores
print(f'코사인 유사도 계산의 결과 : \n{df}')

print('-'*80)

# score값이 최대인 인덱스 추출 -> idxmax() -> series자료형의 전용함수
idx = df.loc[:,'score'].idxmax()
print(f'사용자 입력 문장과 가장 유사한 예상 질문의 인덱스 : \n{idx}')
print('-'*80)

# 코사인 유사도 값이 가장 큰 예상 질문에 대한 답변 추출
# answer = df.loc[idx,'Q':'A']
answer = df.loc[idx,'A']
print(f'챗봇의 답변 : \n{answer}')

# max_score = df.loc[idx,:]
# print(max_score)

100%|██████████| 11823/11823 [00:00<00:00, 111257.52it/s]

코사인 유사도 계산의 결과 : 
                             Q                         A  label  \
0                       12시 땡!                하루가 또 가네요.      0   
1                  1지망 학교 떨어졌어                 위로해 드립니다.      0   
2                 3박4일 놀러가고 싶다               여행은 언제나 좋죠.      0   
3              3박4일 정도 놀러가고 싶다               여행은 언제나 좋죠.      0   
4                      PPL 심하네                눈살이 찌푸려지죠.      0   
...                        ...                       ...    ...   
11818           훔쳐보는 것도 눈치 보임.        티가 나니까 눈치가 보이는 거죠!      2   
11819           훔쳐보는 것도 눈치 보임.             훔쳐보는 거 티나나봐요.      2   
11820              흑기사 해주는 짝남.                    설렜겠어요.      2   
11821  힘든 연애 좋은 연애라는게 무슨 차이일까?  잘 헤어질 수 있는 사이 여부인 거 같아요.      2   
11822               힘들어서 결혼할까봐        도피성 결혼은 하지 않길 바라요.      2   

                                               embedding     score  
0      [-0.13874352, 0.015109689, 0.5284365, 0.721492... -0.021819  
1      [0.08192271, 0.30836907, 0.27680

In [10]:
### 사용자 입력
text = input()
print(text)

사용자 입력
사용자 입력


In [36]:
### 사용자 질문에 대한 응답 구현 함수 정의 -> chatbot 개발 완성

def chatbot(question):
    # 사용자 질문 -> 문장 임베딩 벡터 생성
    user_embedding = model.encode(question)
    # 사용자 질문과 예상 질문간의 코사인 유사도 계산
    scores=[]
    for embedding in df.loc[:,'embedding']:
        score = cos_sim(user_embedding,embedding)
        scores.append(score)
    # score 컬럼 생성 -> 코사인 유사도 값 저장
    df.loc[:,'score'] = scores
    # 코사인 유사도 값 최대인 인덱스 추출 -> 답변 추출
    idx = df.loc[:,'score'].idxmax()
    # answer = df.loc[idx,'Q':'A']
    answer = df.loc[idx,'A']
    return answer

## ChatBot 실행

In [37]:
### 성능 테스트(1)

# 사용자 입력
text = input()

# 챗봇의 답변
answer = chatbot(question=text)
print(answer)

출근하기 싫어
직장 스트레스가 심한가봐요.


In [35]:
### 성능 테스트(2)

# 사용자 입력
text = input()

# 챗봇의 답변
answer = chatbot(question=text)
print(answer)

수업듣기싫어
Q       공부하기 싫어
A    잠시 쉬어도 돼요.
Name: 229, dtype: object


In [38]:
### 성능 테스트(3)

# 사용자 입력
text = input()

# 챗봇의 답변
answer = chatbot(question=text)
print(answer)

나 아파
병원에 가야죠.
